## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-17-14-53-22 +0000,bbc,Fast-track deportations to be announced as par...,https://www.bbc.com/news/articles/cn97ryvelz0o...
1,2025-11-17-14-51-06 +0000,nyt,An Escape From India’s Air Pollution for Those...,https://www.nytimes.com/2025/11/14/business/in...
2,2025-11-17-14-46-19 +0000,nypost,Man charged with killing 4 in Tennessee to mak...,https://nypost.com/2025/11/17/us-news/man-char...
3,2025-11-17-14-40-08 +0000,nyt,"Drawing Religion Minister’s Ire, Music Video i...",https://www.nytimes.com/2025/11/17/world/asia/...
4,2025-11-17-14-37-00 +0000,wsj,Canada Inflation Eases to 2.2% in October,https://www.wsj.com/economy/central-banking/ca...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2312/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
83,trump,35
285,epstein,13
163,new,12
286,files,11
173,up,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
128,2025-11-17-04-47-00 +0000,wsj,Trump threw in the towel on dissuading House R...,https://www.wsj.com/politics/policy/trump-play...,102
140,2025-11-17-02-44-24 +0000,nypost,Trump says House Republicans should vote to re...,https://nypost.com/2025/11/16/us-news/trump-sa...,85
234,2025-11-16-16-58-43 +0000,nypost,MTG suggests Israel or ‘other foreign gov’t’ m...,https://nypost.com/2025/11/16/us-news/mtg-sugg...,85
88,2025-11-17-10-21-07 +0000,nyt,Trump Says House Republicans Should Vote to Re...,https://www.nytimes.com/2025/11/16/us/politics...,80
53,2025-11-17-11-56-04 +0000,bbc,Trump calls on House Republicans to vote to re...,https://www.bbc.com/news/articles/cvgv653v1vjo...,78


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
128,102,2025-11-17-04-47-00 +0000,wsj,Trump threw in the towel on dissuading House R...,https://www.wsj.com/politics/policy/trump-play...
10,40,2025-11-17-14-27-00 +0000,wsj,Former Bangladesh Leader Sheikh Hasina Sentenc...,https://www.wsj.com/world/asia/former-banglade...
176,38,2025-11-16-23-33-06 +0000,nypost,Vile antisemitic graffiti scrawled on Mexico’s...,https://nypost.com/2025/11/16/world-news/antis...
192,37,2025-11-16-21-02-08 +0000,latimes,Body of 7-year-old girl is found off Big Sur; ...,https://www.latimes.com/california/story/2025-...
169,36,2025-11-17-00-00-21 +0000,bbc,Formula 1 tech used to rev-up train wi-fi spee...,https://www.bbc.com/news/articles/cqjwkdez71ro...
102,29,2025-11-17-09-32-23 +0000,bbc,Dozens of Indian pilgrims killed in bus accide...,https://www.bbc.com/news/articles/cvg7dme9en0o...
142,29,2025-11-17-02-30-26 +0000,nypost,Trump says Venezuela would ‘like to talk’ — as...,https://nypost.com/2025/11/16/world-news/trump...
233,29,2025-11-16-17-00-00 +0000,wsj,Over 60% of Americans are comfortable with how...,https://www.wsj.com/politics/policy/economic-o...
49,28,2025-11-17-12-00-45 +0000,nypost,Air Force vet mysteriously found dead in mail ...,https://nypost.com/2025/11/17/us-news/air-forc...
221,27,2025-11-16-18-36-27 +0000,nyt,UN Climate Summit in Brazil Brings Tourism and...,https://www.nytimes.com/2025/11/16/world/ameri...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
